In [0]:
dbutils.library.restartPython() 

#### Connect to storage & load libraries

In [0]:
ServicePrincipalID = dbutils.secrets.get(scope="data-bricks", key="SPID")
ServicePrincipalKey = dbutils.secrets.get(scope="data-bricks", key="SPKey")
DirectoryID = dbutils.secrets.get(scope="data-bricks", key="DirectoryID")

Directory = f"https://login.microsoftonline.com/{DirectoryID}/oauth2/token"

container_name = 'my-csv'
storage_account_name = 'mainstorageaccountv2'

url=f'abfss://{container_name}@{storage_account_name}.dfs.core.windows.net'
mnt_path = '/mnt/test'
table_path = 'tables'

configs = {
  "fs.azure.account.auth.type": "OAuth",
  "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
  "fs.azure.account.oauth2.client.id": ServicePrincipalID,
  "fs.azure.account.oauth2.client.secret": ServicePrincipalKey,
  "fs.azure.account.oauth2.client.endpoint": Directory,
  "fs.azure.createRemoteFileSystemDuringInitialization": "false"
}

dbutils.fs.mount(
  source = url,
  mount_point = mnt_path,
  extra_configs = configs
)

Out[46]: True

In [0]:
# Import Libraries from pyspark
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, col, date_format, to_date, add_months, coalesce,sum,substring,date_add,expr,datediff
from datetime import *

In [0]:
dbutils.fs.ls(mnt_path)

Out[54]: [FileInfo(path='dbfs:/mnt/test/input/', name='input/', size=0),
 FileInfo(path='dbfs:/mnt/test/market_data.csv', name='market_data.csv', size=68870),
 FileInfo(path='dbfs:/mnt/test/output/', name='output/', size=0),
 FileInfo(path='dbfs:/mnt/test/ufo.csv', name='ufo.csv', size=2957462)]

#### Create database in Databricks

In [0]:
db_name = 'test'

spark.sql(f"""CREATE DATABASE IF NOT EXISTS {db_name} COMMENT 'This is for test purposes' LOCATION '/user';""")

spark.sql(f'DESCRIBE DATABASE EXTENDED {db_name}').show(truncate=False)

+-------------------------+--------------------------+
database_description_item|database_description_value|
+-------------------------+--------------------------+
Database Name |test |
Comment |This is for test purposes |
Location |dbfs:/user |
Owner |root |
Properties | |
+-------------------------+--------------------------+

#### Drop database (just in case)

In [0]:
#spark.sql(f"""DROP DATABASE IF EXISTS {db_name};""")

#### Read csv from storage

In [0]:
# source https://sdm.lbl.gov/fastbit/data/samples.html
file_path = f'{mnt_path}/input/star2002-full.csv'
db_table = f'{db_name}.star2000'

In [0]:
schema = StructType([
  StructField('antiNucleus', IntegerType(), nullable=False),
  StructField('eventFile', DoubleType(), nullable=True),
  StructField('eventNumber', DoubleType(), nullable=True),
  StructField('eventTime', DoubleType(), nullable=True),
  StructField('histFile', DoubleType(), nullable=True),
  StructField('multiplicity', DoubleType(), nullable=True),
  StructField('NaboveLb', DoubleType(), nullable=True),
  StructField('NbelowLb', DoubleType(), nullable=True),
  StructField('NLb', DoubleType(), nullable=True),
  StructField('primaryTracks', DoubleType(), nullable=True),
  StructField('prodTime', DoubleType(), nullable=True),
  StructField('Pt', FloatType(), nullable=True),
  StructField('runNumber', FloatType(), nullable=True),
  StructField('vertexX', FloatType(), nullable=True),
  StructField('vertexY', FloatType(), nullable=True),
  StructField('vertexZ', FloatType(), nullable=True)
])

In [0]:
df_csv = (
  spark.read.format("csv")
  .option('header','false')
  .option('delimiter', ',')
  #.option('inferSchema', 'true')
  #.option('format', '%i')
  .schema(schema)
  .load(file_path)
)

In [0]:
df_csv.na.drop()

Out[15]: DataFrame[antiNucleus: int, eventFile: double, eventNumber: double, eventTime: double, histFile: double, multiplicity: double, NaboveLb: double, NbelowLb: double, NLb: double, primaryTracks: double, prodTime: double, Pt: float, runNumber: float, vertexX: float, vertexY: float, vertexZ: float]

In [0]:
df_csv.printSchema()

root
-- antiNucleus: integer (nullable = true)
-- eventFile: double (nullable = true)
-- eventNumber: double (nullable = true)
-- eventTime: double (nullable = true)
-- histFile: double (nullable = true)
-- multiplicity: double (nullable = true)
-- NaboveLb: double (nullable = true)
-- NbelowLb: double (nullable = true)
-- NLb: double (nullable = true)
-- primaryTracks: double (nullable = true)
-- prodTime: double (nullable = true)
-- Pt: float (nullable = true)
-- runNumber: float (nullable = true)
-- vertexX: float (nullable = true)
-- vertexY: float (nullable = true)
-- vertexZ: float (nullable = true)

In [0]:
df_csv.show(5)

+-----------+---------+-----------+-----------------+---------+------------+--------+--------+------+-------------+-----------------+---------+---------+-----------+----------+---------+
antiNucleus|eventFile|eventNumber| eventTime| histFile|multiplicity|NaboveLb|NbelowLb| NLb|primaryTracks| prodTime| Pt|runNumber| vertexX| vertexY| vertexZ|
+-----------+---------+-----------+-----------------+---------+------------+--------+--------+------+-------------+-----------------+---------+---------+-----------+----------+---------+
 1|1613423.0| 807.0|2.0011015222604E7|1613424.0| 4518.0| 0.0| 0.0| 654.0| 1395.0|2.0011204114951E7|10.955403|2288071.0|-0.28820264|0.40731233|10.559091|
 1|1613423.0| 808.0|2.0011015222604E7|1613424.0| 886.0| 0.0| 0.0| 61.0| 371.0|2.0011204115313E7|23.326479|2288071.0|-0.24732992| 0.4559155|57.810596|
 1|1613423.0| 809.0|2.0011015222604E7|1613424.0| 638.0| 0.0| 0.0| 7.0| 121.0|2.0011204115355E7|2.4442992|2288071.0|-0.39096078|0.58953387|167.75714|
 4|1613423.0| 810.0|2.0011015222604E7|1613424.0| 4259.0| 0.0| 0.0|1024.0| 1302.0| 2.001120411543E7| 9.521868|2288071.0|-0.29015437|0.44602653| 8.644362|
 5|1613423.0| 811.0|2.0011015222604E7|1613424.0| 3673.0| 1.0| 0.0| 592.0| 1246.0|2.0011204115721E7|13.560424|2288071.0| -0.2574184|0.41968927| 29.02236|
+-----------+---------+-----------+-----------------+---------+------------+--------+--------+------+-------------+-----------------+---------+---------+-----------+----------+---------+
only showing top 5 rows

#### Repartition

In [0]:
#df_csv = df_csv.repartition("antiNucleus")

In [0]:
partition_by = 'antiNucleus'

In [0]:
#df = sqlContext.createDataFrame(df_csv.collect(), schema=schema)

java.lang.Exception: Could not stop computations
	at com.databricks.backend.daemon.driver.DriverClient.$anonfun$poll$3(DriverClient.scala:416)
	at scala.concurrent.Future.$anonfun$flatMap$1(Future.scala:307)
	at scala.concurrent.impl.Promise.$anonfun$transformWith$1(Promise.scala:41)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at com.databricks.threading.NamedExecutor$$anon$2.$anonfun$run$1(NamedExecutor.scala:345)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:240)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:235)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:232)
	at com.databricks.threading.NamedExecutor.withAttributionContext(NamedExecutor.scala:275)
	at com.databricks.threading.NamedExecutor$$anon$2.run(NamedExecutor.scala:345)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:834)

#### Create dataframe and write it to storage

In [0]:
table_file_path = f'{mnt_path}/{table_path}/{db_table}'

In [0]:
(df_csv
 .write
 .partitionBy(partition_by)
 .format('delta')
 .mode('overwrite')
 .option('mergeSchema','true')
 .option('overwriteSchema', 'true') # if table already exists and you changed the type of some column
 .save(table_file_path)
)

In [0]:
#spark.sql("""drop table if exists {}""".format(db_table))

Out[25]: DataFrame[]

In [0]:
#df_csv.filter(df_csv.antiNucleus.isNull()).show()

+-----------+---------+-----------+---------+--------+------------+--------+--------+---+-------------+--------+---+---------+-------+-------+-------+
antiNucleus|eventFile|eventNumber|eventTime|histFile|multiplicity|NaboveLb|NbelowLb|NLb|primaryTracks|prodTime| Pt|runNumber|vertexX|vertexY|vertexZ|
+-----------+---------+-----------+---------+--------+------------+--------+--------+---+-------------+--------+---+---------+-------+-------+-------+
+-----------+---------+-----------+---------+--------+------------+--------+--------+---+-------------+--------+---+---------+-------+-------+-------+

In [0]:
# Create table in Databricks

spark.sql("""create table if not exists {} (
antiNucleus	INTEGER ,
eventFile	DOUBLE,
eventNumber	DOUBLE,
eventTime	DOUBLE,
histFile	DOUBLE,
multiplicity	DOUBLE,
NaboveLb	DOUBLE,
NbelowLb	DOUBLE,
NLb		DOUBLE,
primaryTracks	DOUBLE,
prodTime	DOUBLE,
Pt		FLOAT,
runNumber	FLOAT,
vertexX		FLOAT,
vertexY		FLOAT,
vertexZ		FLOAT
) 
using delta location '{}' 
partitioned by ({})""".format(db_table, table_file_path, partition_by))

Out[36]: DataFrame[]

In [0]:
df = spark.read.table(db_table).where(col('antiNucleus') == 1)
df.show(5, truncate=True)

+-----------+---------+-----------+-----------------+---------+------------+--------+--------+-----+-------------+-----------------+---------+---------+-----------+----------+---------+
antiNucleus|eventFile|eventNumber| eventTime| histFile|multiplicity|NaboveLb|NbelowLb| NLb|primaryTracks| prodTime| Pt|runNumber| vertexX| vertexY| vertexZ|
+-----------+---------+-----------+-----------------+---------+------------+--------+--------+-----+-------------+-----------------+---------+---------+-----------+----------+---------+
 1|1613423.0| 807.0|2.0011015222604E7|1613424.0| 4518.0| 0.0| 0.0|654.0| 1395.0|2.0011204114951E7|10.955403|2288071.0|-0.28820264|0.40731233|10.559091|
 1|1613423.0| 808.0|2.0011015222604E7|1613424.0| 886.0| 0.0| 0.0| 61.0| 371.0|2.0011204115313E7|23.326479|2288071.0|-0.24732992| 0.4559155|57.810596|
 1|1613423.0| 809.0|2.0011015222604E7|1613424.0| 638.0| 0.0| 0.0| 7.0| 121.0|2.0011204115355E7|2.4442992|2288071.0|-0.39096078|0.58953387|167.75714|
 1|1613423.0| 813.0|2.0011015222604E7|1613424.0| 1115.0| 0.0| 0.0| 27.0| 384.0| 2.001120412025E7|20.839916|2288071.0|-0.23799685|0.46584174|7.9682536|
 1|1613423.0| 817.0|2.0011015222604E7|1613424.0| 170.0| 0.0| 0.0| 1.0| 57.0|2.0011204120734E7|2.6370063|2288071.0| -0.2592082|0.32376853|94.953995|
+-----------+---------+-----------+-----------------+---------+------------+--------+--------+-----+-------------+-----------------+---------+---------+-----------+----------+---------+
only showing top 5 rows

In [0]:
display(df.describe())

summary,antiNucleus,eventFile,eventNumber,eventTime,histFile,multiplicity,NaboveLb,NbelowLb,NLb,primaryTracks,prodTime,Pt,runNumber,vertexX,vertexY,vertexZ
count,1914455,1914455,1914455,1914455,1914455,1914455,1914455,1914455,1914455,1914455,1914455,1914455,1914455,1914455,1914455,1914455
mean,1.0,1398887.5412511656,26193.61024573573,2.0012577111241035E7,1399649.1422164533,3406.6967795012156,0.747292571515131,0.019231060536810736,501.68830554909886,1188.7437871352422,2.0020641974674325E7,230.36955158491918,2434862.3178868135,-0.17763027174575308,0.43809678647735845,0.2906302617067617
stddev,0.0,568151.230218667,44757.20877800337,5209.264228815674,567241.0966940541,1978.0757528154882,1.6046114571584256,0.17203304241935194,396.2549455636793,671.8925955578408,3648.319784977068,20630.417000709564,472822.0789522565,0.8118345462460678,0.5767854198871102,33.099798574748995
min,1,0.0,1.0,2.0010911035914E7,0.0,1.0,0.0,0.0,0.0,1.0,2.0011028010128E7,0.0,2253050.0,-48.705025,-38.519547,-502.69006
max,1,2761475.0,586066.0,2.0030208180455E7,2761480.0,10513.0,216.0,8.0,3559.0,2929.0,2.0030322004355E7,2.4420602E7,4039022.0,39.67056,44.209034,622.54144


#### Clear everything

In [0]:
spark.sql("""drop table if exists {}""".format(db_table))

Out[49]: DataFrame[]

In [0]:
spark.sql(f"""DROP DATABASE IF EXISTS {db_name};""")

Out[50]: DataFrame[]

In [0]:
dbutils.fs.rm(table_file_path, recurse=True)

Out[51]: True

In [0]:
dbutils.fs.ls(mnt_path)

Out[53]: [FileInfo(path='dbfs:/mnt/test/input/', name='input/', size=0),
 FileInfo(path='dbfs:/mnt/test/market_data.csv', name='market_data.csv', size=68870),
 FileInfo(path='dbfs:/mnt/test/output/', name='output/', size=0),
 FileInfo(path='dbfs:/mnt/test/ufo.csv', name='ufo.csv', size=2957462)]

In [0]:
dbutils.fs.unmount(mnt_path) 

/mnt/test has been unmounted.
Out[45]: True

#### End of the notebook